# Exploring SPMV

First, include some libraries

In [ ]:
# Run boilerplate code to set up environment

%run ../prelude.py

#  Create Random Matrix


In [ ]:
import random 

random.seed(0)
def GenerateRandomGraph(numVertex=8, sparsity=25, symmetric=True):
    adjMatrix = [[0 for d in range(numVertex)] for s in range(numVertex)]
    
    genNode = int(numVertex * numVertex * sparsity / 100)
    
    for n in range(genNode):
        while True:
            src = random.randrange(numVertex)
            dst = random.randrange(numVertex)
            if src != dst:
                break
                
        adjMatrix[src][dst] = 1
        if symmetric:
            adjMatrix[dst][src] = 1
    
    return adjMatrix

## Graph Inputs

In [ ]:
#
# Function to create graph inputs
#


# Create inputs based on the slides that Aamer used to do the initial SPMV mapping on Symphony
def create_inputs(display=True, numNode=8):
    # Adjacency matrix - Ranks "S" (source) and "D" (destination)

#    matrix = GenerateRandomGraph(numNode, 25)
#    adjMat = Tensor.fromUncompressed([ "S", "D"], matrix)

    adjMat = Tensor.fromUncompressed([ "S", "D"],
                                [ [ 0, 1, 0, 0, 1, 0, 1, 0 ],
                                  [ 1, 0, 0, 0, 1, 1, 0, 1 ],
                                  [ 0, 0, 0, 0, 0, 0, 0, 0 ],
                                  [ 0, 0, 0, 0, 0, 0, 0, 0 ],
                                  [ 1, 1, 0, 0, 0, 0, 0, 1 ],
                                  [ 0, 1, 0, 0, 0, 0, 1, 1 ],
                                  [ 1, 0, 0, 0, 0, 1, 0, 1 ],
                                  [ 0, 1, 0, 0, 1, 1, 1, 0 ] ])

    # Vector
    srcData = Tensor.fromUncompressed([ "S" ], [ 1 for s in range(numNode) ])

    # Vector
    dstData = Tensor.fromUncompressed([ "D" ], [ 0 for d in range(numNode) ])
    
    print("Created Adjacency Matrix")
    if display:
        displayTensor(adjMat)

    print("Created SrcData")
    if display:
        displayTensor(srcData)

    print("Created DstData")
    if display:
        displayTensor(dstData)

    return (dstData, adjMat, srcData)



In [ ]:
# importing array module 
import array as arr 

# Create inputs
(dstData, adjMat, srcData) = create_inputs(False)

# Create CSF
tileSizes = arr.array('l', [2,2])
createCSF4( adjMat, tileSizes)

# Get root fibers
#adjMat_s  = adjMat.getRoot()
#srcData_v = srcData.getRoot()
#dstData_v = dstData.getRoot()

# Tile the adjacency matrix in the source dimension 
#print("Tiling the adjacency matrix in coordinate space uniformly with tile size=2")
#adjMat_tiled_s1 = adjMat_s.splitUniform(2)

# display the tiled tensor
#displayTensor(adjMat_tiled_s1)

# Tile the srcData vector in the src dimension 
#print("Tiling the srcData vector in coordinate space uniformly with tile size=2")
#srcData_tiled_v1 = srcData_v.splitUniform(2)
#displayTensor(srcData_tiled_v1)

# Tile the dstData vector in the dst dimension 
#print("Tiling the dstData vector in coordinate space uniformly with tile size=2")
#dstData_tiled_v1 = dstData_v.splitUniform(2)
#displayTensor(dstData_tiled_v1)

# create a tensor frame
#canvas = createCanvas(adjMat_tiled_s1, srcData_tiled_v1, dstData)

# perform the tiled src stationary SPMV
#for s1, (adjMat_tiled_s0, srcData_tiled_v0) in (adjMat_tiled_s1 & srcData_tiled_v1):
#    for s0, (adjMat_tiled_d, srcData_tiled_val) in (adjMat_tiled_s0 & srcData_tiled_v0):
#        for d, (dstData_ref, adjMat_tiled_val) in (dstData_v << adjMat_tiled_d):
#            dstData_ref += adjMat_tiled_val * srcData_tiled_val
#            addFrame(canvas, [s1,s0,d], [s1,s0], [d])

# display the canvas animation
#displayCanvas(canvas)


# Naive SPMV - source stationary (push)

This version traverses all neighbors of each source node.

In [ ]:
V  = 8

# Create inputs
(dstData, adjMat, srcData) = create_inputs(False, V)

# Get root fibers
adjMat_s  = adjMat.getRoot()
srcData_v = srcData.getRoot()
dstData_v = dstData.getRoot()

#print("Displaying Graph")
displayGraph(adjMat_s)

#print("Displaying Fiber Tree Representation of Adjacency Matrix")
displayTensor(adjMat)

#print("Displaying Fiber Tree Representation of Vector")
#displayTensor(srcData)

# create a tensor frame
canvas = createCanvas(adjMat, srcData, dstData)

# Perform the src stationary SPMV
print("Running SRC stationary SPMV:")

for s, (adjMat_d, srcData_val) in (adjMat_s & srcData_v):
    for d, (dstData_ref, adjMat_val), in (dstData_v << adjMat_d):
        dstData_ref += adjMat_val * srcData_val
        addFrame(canvas, [s,d], [s], [d])

# display the canvas animation
displayCanvas(canvas)



# Naive SPMV - destination stationary (pull)

This version traverses all neighbors of each destination node.

In [ ]:
V  = 8

# Create inputs
(dstData, adjMat, srcData) = create_inputs(False, V)

# Get root fibers
adjMat_s  = adjMat.getRoot()
srcData_v = srcData.getRoot()
dstData_v = dstData.getRoot()

#print("Displaying Graph")
#displayGraph(adjMat_s)

# For destination stationary we need to swap the matrix
adjMat_transpose_d = adjMat_s.swapRanks();

#print("Displaying Transposed Graph")
#displayGraph(adjMat_transpose_d)

# create the actual transposed tensor in memory 
print("Creating Transpose of Typical Adjacency Matrix to Perform Dst Stationary Traversal:")
adjMat_transposed = Tensor.fromFiber(["D", "S"], adjMat_transpose_d)

#print("Displaying Fiber Tree Representation of Transposed Adjacency Matrix")
#displayTensor(adjMat_transposed)

#print("Displaying Fiber Tree Representation of srcData")
#displayTensor(srcData)

#print("Displaying Fiber Tree Representation of dstData")
#displayTensor(dstData)

# create the new canvas with transposed adjacency matrix
canvas = createCanvas(adjMat_transposed, srcData, dstData)

# Perform the dst stationary SPMV
print("Running DST stationary SPMV:")

# Perform the destination stationary SPMV
for d, (dstData_ref, adjMat_transpose_s) in (dstData_v << adjMat_transpose_d):
    for s, (adjMat_transpose_val, srcData_val), in (adjMat_transpose_s & srcData_v):
        dstData_ref += adjMat_transpose_val * srcData_val
        addFrame(canvas, [d,s], [s], [d])

# display the canvas animation
displayCanvas(canvas)



# 1D -- Spatially Tiled SPMV - src stationary (push)

This version traverses all neighbors of each source node.

In [ ]:
# Create inputs
(dstData, adjMat, srcData) = create_inputs(False)

# Get root fibers
adjMat_s  = adjMat.getRoot()
srcData_v = srcData.getRoot()
dstData_v = dstData.getRoot()

# Tile the adjacency matrix in the source dimension 
print("Tiling the adjacency matrix in coordinate space uniformly with tile size=2")
adjMat_tiled_s1 = adjMat_s.splitUniform(2)
displayTensor(adjMat_tiled_s1)

# Tile the srcData vector in the source dimension 
print("Tiling the srcData vector in coordinate space uniformly with tile size=2")
srcData_tiled_v1 = srcData_v.splitUniform(2)
#displayTensor(srcData_tiled_v1)

# create a tensor frame
canvas = createCanvas(adjMat_tiled_s1, srcData_tiled_v1, dstData)

# Perform the src stationary SPMV
print("Running Tiled SRC stationary SPMV:")

# perform the tiled src stationary SPMV
for s1, (adjMat_tiled_s0, srcData_tiled_v0) in (adjMat_tiled_s1 & srcData_tiled_v1):
    for s0, (adjMat_tiled_d, srcData_tiled_val) in (adjMat_tiled_s0 & srcData_tiled_v0):
        for d, (dstData_ref, adjMat_tiled_val) in (dstData_v << adjMat_tiled_d):
            dstData_ref += adjMat_tiled_val * srcData_tiled_val
            addFrame(canvas, [s1,s0,d], [s1,s0], [d])

# display the canvas animation
displayCanvas(canvas)


# 2D -- Spatially Tiled SPMV - src stationary (push)  -- Work In Progress

This version traverses all neighbors of each source node.

# 1D -- Spatially Tiled SPMV - outer src, inner dst stationary (pull)

In [ ]:
# Create inputs
(dstData, adjMat, srcData) = create_inputs(False)

# Get root fibers
adjMat_s  = adjMat.getRoot()
srcData_v = srcData.getRoot()
dstData_v = dstData.getRoot()

# Tile the adjacency matrix in the source dimension 
print("Tiling the adjacency matrix in coordinate space uniformly with tile size=2")
#adjMat_tiled_s1 = adjMat_s.splitUniform(2)
#displayTensor(adjMat_tiled_s1)

displayTensor(adjMat_s)
adjMat_tiled_s1 = adjMat_s.splitUniform(2)
displayTensor(adjMat_tiled_s1)

adjMat_tiled_s1.swapRanksBelow()
displayTensor(adjMat_tiled_s1)

# Tile the srcData vector in the source dimension 
print("Tiling the srcData vector in coordinate space uniformly with tile size=2")
srcData_tiled_v1 = srcData_v.splitUniform(2)
displayTensor(srcData_tiled_v1)

# create a tensor frame
canvas = createCanvas(adjMat_tiled_s1, srcData_tiled_v1, dstData)

# Perform the src stationary SPMV
print("Running Tiled SRC stationary SPMV:")

# perform the src tiled dst stationary SPMV
for s1, (adjMat_tiled_d, srcData_tiled_v0) in (adjMat_tiled_s1 & srcData_tiled_v1):
    for d, (dstData_ref, adjMat_tiled_s0) in (dstData_v << adjMat_tiled_d):
        for s0, (adjMat_tiled_val, src_data_tiled_val) in (adjMat_tiled_s0 & srcData_tiled_v0):
            dstData_ref += adjMat_tiled_val * srcData_tiled_val
            addFrame(canvas, [s1,s0,d], [s1,s0], [d])

# display the canvas animation
displayCanvas(canvas)

# 2D -- Spatially Tiled SPMV - outer src, inner src stationary (push)

In [ ]:
# Create inputs
(dstData, adjMat, srcData) = create_inputs(False)

# Get root fibers
adjMat_s  = adjMat.getRoot()
srcData_v = srcData.getRoot()
dstData_v = dstData.getRoot()

# Tile the adjacency matrix in the source dimension 
print("Tiling the adjacency matrix in coordinate space uniformly with tile size=2")
#adjMat_tiled_s1 = adjMat_s.splitUniform(2)
#displayTensor(adjMat_tiled_s1)

displayTensor(adjMat_s)
adjMat_tiled_s1s0d = adjMat_s.splitUniform(2)
displayTensor(adjMat_tiled_s1s0d)

adjMat_tiled_s1s0d.splitUniformBelow(2, depth=1)
adjMat_tiled_s1s0d1d0 = adjMat_tiled_s1s0d
displayTensor(adjMat_tiled_s1s0d1d0)

adjMat_tiled_s1s0d1d0.swapRanksBelow()
adjMat_tiled_s1d1s0d0 = adjMat_tiled_s1s0d1d0
displayTensor(adjMat_tiled_s1d1s0d0)

# Tile the srcData vector in the src dimension 
print("Tiling the srcData vector in coordinate space uniformly with tile size=2")
srcData_tiled_v1 = srcData_v.splitUniform(2)
displayTensor(srcData_tiled_v1)

# Tile the dstData vector in the dst dimension 
print("Tiling the dstData vector in coordinate space uniformly with tile size=2")
dstData_tiled_v1 = dstData_v.splitUniform(2)
displayTensor(dstData_tiled_v1)


# create a tensor frame
canvas = createCanvas(adjMat_tiled_s1d1s0d0, srcData_tiled_v1, dstData_tiled_v1)

# Perform the src stationary SPMV
print("Running 2D Tiled SRC stationary SPMV:")

# perform the 2D tiled src tiled dst stationary SPMV
for s1, (adjMat_tiled_d1, srcData_tiled_v0) in (adjMat_tiled_s1 & srcData_tiled_v1):
    for d1, (dstData_tiled_v0, adjMat_tiled_s0) in (dstData_tiled_v1 << adjMat_tiled_d1):
        for s0, (adjMat_tiled_d0, src_data_tiled_val) in (adjMat_tiled_s0 & srcData_tiled_v0):
            for d0, ( _, adjMat_tiled_val) in ( adjMat_tiled_d0):
                dstData_ref += adjMat_tiled_val * srcData_tiled_val
                addFrame(canvas, [s1,d1, s0,d0], [s1,s0], [d1, d0])

# display the canvas animation
displayCanvas(canvas)

# Online 2D Tiled SPMV

In [ ]:
#Create inputs
(dstData, adjMat, srcData) = create_inputs(False)

# Get root fibers
adjMat_s  = adjMat.getRoot()
srcData_v = srcData.getRoot()
dstData_v = dstData.getRoot()

# create a tensor frame
canvas = createCanvas(adjMat, srcData, dstData)

V  = adjMat.getRoot().maxCoord() + 1
S0 = 2
D0 = 4
    
S1 = int((V/S0,V/S0+1) [V%S0])
D1 = int((V/D0,V/D0+1) [V%D0])

print("V  = ", V)
    
print("S0 = ", S0)
print("D0 = ", D0)
    
print("S1 = ", S1)
print("D1 = ", D1)
    
displayTensor(adjMat)
    
for s1 in range(S1):
    for d1 in range(D1):
        for s, (adjMat_d, srcData_val) in (adjMat_s & srcData_v):
            s1_of_s = int(s/S0)          
            if s1_of_s == s1:
                for d, (dstData_ref, adjMat_val), in (dstData_v << adjMat_d):
                    d1_of_d = int(d/D0)
                    if d1_of_d == d1:
                        dstData_ref += adjMat_val * srcData_val
                        addFrame(canvas, [s,d], [s], [d])
            

# display the canvas animation
displayCanvas(canvas)

# Online 1D SRC Tiled SPMV using S,D FiberTree

In [ ]:
V  = 8

#Create inputs
(dstData, adjMat, srcData) = create_inputs(False, V)

# Get root fibers
adjMat_s  = adjMat.getRoot()
srcData_v = srcData.getRoot()
dstData_v = dstData.getRoot()



S0 = V
D0 = 1


print("V  = ", V)
    
print("S0 = ", S0)
print("D0 = ", D0)

S1 = int((V/S0+1,V/S0) [(V%S0) == 0])
D1 = int((V/D0+1,V/D0) [(V%D0) == 0])

    
print("S1 = ", S1)
print("D1 = ", D1)


# create a tensor frame
canvas = createCanvas(adjMat, srcData, dstData)

for s1 in range(S1):
    for d1 in range(D1):
        for s, (adjMat_d, srcData_val) in (adjMat_s & srcData_v):
            s1_of_s = int(s/S0)          
            if s1_of_s == s1:
                for d, (dstData_ref, adjMat_val), in (dstData_v << adjMat_d):
                    d1_of_d = int(d/D0)
                    if d1_of_d == d1:
                        dstData_ref += adjMat_val * srcData_val
                        addFrame(canvas, [s,d], [s], [d])
                                  
# display the canvas animation
print("Online Tiling")
displayCanvas(canvas)

print("Pre-Processed")
adjMat_s = adjMat.getRoot()
adjMat_s.splitUniformBelow(2)
adjMat_swap = adjMat_s.swapRanks()
displayTensor(adjMat_swap)


# Online 1D SRC Tiled SPMV using D,S FiberTree

In [ ]:
V  = 8

#Create inputs
(dstData, adjMat, srcData) = create_inputs(False, V)

# Get root fibers
adjMat_s  = adjMat.getRoot()
srcData_v = srcData.getRoot()
dstData_v = dstData.getRoot()

print("Fiber Tree (S,D)")
displayTensor(adjMat_s)

displayGraph(adjMat_s)


adjMat_d = adjMat_s.swapRanks()

print("Fiber Tree (D,S)")
displayTensor(adjMat_d)



S0 = V
D0 = 2


print("V  = ", V)
    
print("S0 = ", S0)
print("D0 = ", D0)

S1 = int((V/S0+1,V/S0) [(V%S0) == 0])
D1 = int((V/D0+1,V/D0) [(V%D0) == 0])

    
print("S1 = ", S1)
print("D1 = ", D1)


# create a tensor frame
canvas_sd = createCanvas(adjMat, srcData, dstData)

#for s1 in range(S1):
#    for d1 in range(D1):
#        for s, (adjMat_d, srcData_val) in (adjMat_s & srcData_v):
#            s1_of_s = int(s/S0)          
#            if s1_of_s == s1:
#                for d, (dstData_ref, adjMat_val), in (dstData_v << adjMat_d):
#                    d1_of_d = int(d/D0)
#                    if d1_of_d == d1:
#                        dstData_ref += adjMat_val * srcData_val
#                        canvas_sd.addFrame([s,d], [s], [d])



# display the canvas animation
print("Online Tiling")
displayCanvas(canvas_sd)

adjMat_s  = adjMat.getRoot()
adjMat_d = adjMat_s.swapRanks()

dstData = Tensor.fromUncompressed([ "V" ], [ 0 for d in range(V) ])
dstData_v = dstData.getRoot()

# create a tensor frame
canvas_ds = createCanvas(adjMat_d, srcData, dstData)

for s1 in range(S1):
    for d1 in range(D1):
        for d, (dstData_ref, adjMat_s), in (dstData_v << adjMat_d):
            d1_of_d = int(d/D0)
            if d1_of_d == d1:
                for s, (adjMat_val, srcData_val) in (adjMat_s & srcData_v):
                    s1_of_s = int(s/S0)          
                    if s1_of_s == s1:
                        dstData_ref += adjMat_val * srcData_val
                        canvas_ds.addFrame([d,s], [s], [d])
                                  
# display the canvas animation
print("Online Tiling")
displayCanvas(canvas_ds)

print("Pre-Processed")
adjMat_s = adjMat.getRoot()
adjMat_s.splitUniformBelow(2)
adjMat_swap = adjMat_s.swapRanks()
displayTensor(adjMat_swap)
dstData = Tensor(rank_ids=["D1", "D0"])
dstData_v = dstData.getRoot()

canvas_preprocess = createCanvas(adjMat_swap, srcData, dstData)

for d1, (dstData_d0_ref, adjMat_s), in (dstData_v << adjMat_swap):
    for s, (adjMat_d0, srcData_val) in (adjMat_s & srcData_v):
        for d0, (dstData_ref, adjMat_val) in ( dstData_d0_ref << adjMat_d0):
            dstData_ref += adjMat_val * srcData_val
            canvas_preprocess.addFrame([d1,s,d0], [s], [d1,d0])
            
displayCanvas(canvas_preprocess)

print("2D Tiling")
displayTensor(adjMat_swap)
adjMat_swap.splitUniformBelow(2)
displayTensor(adjMat_swap)
adjMat_swap2D = adjMat_swap.swapRanks()
displayTensor(adjMat_swap2D)

